In [ ]:
#import relevant libraries for the data analysis
import pandas as pd
import numpy as np
import folium
import matplotlib as mpl
import matplotlib.pyplot as plt
print('libraries imported')
pd.set_option('display.max_columns', None)

In [ ]:
#import the dataset
website_url='https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv'
Df= pd.read_csv(website_url, sep=',', error_bad_lines=False, index_col=False,dtype='unicode')
Df.head()
Df.tail()

Df.info()

In [ ]:
# To Check the null or NaN in the dataset
missing_data=Df.isnull()
missing_data.head()

for column in missing_data.columns.values.tolist():
    print(column)
    print(missing_data[column].value_counts())
    print("")


In [ ]:
# Drop unrelevant attributes
df_1=Df.drop(['SEVERITYCODE','EXCEPTRSNCODE','EXCEPTRSNDESC','INCKEY',
              'COLDETKEY','REPORTNO','STATUS','INTKEY','SEGLANEKEY','CROSSWALKKEY',
              'HITPARKEDCAR','SDOTCOLNUM'],axis=1)
df_1.dtypes
df_1.rename(columns={'X':'Longitude','Y':'Latitude',
                     'SEVERITYCODE.1':'SEVERITYCODE'},inplace=True)
df_1

In [ ]:
#reset the dtypes of each column in the df_1
df_1['Longitude']=df_1['Longitude'].astype('float')
df_1['Latitude']=df_1['Latitude'].astype('float')
df_1['OBJECTID']=df_1['OBJECTID'].astype('string')
df_1['ADDRTYPE']=df_1['ADDRTYPE'].astype('string')
df_1['LOCATION']=df_1['LOCATION'].astype('string')
df_1['SEVERITYCODE']=df_1['SEVERITYCODE'].astype('string')
df_1['SEVERITYDESC']=df_1['SEVERITYDESC'].astype('string')
df_1['COLLISIONTYPE']=df_1['COLLISIONTYPE'].astype('string')
df_1['PERSONCOUNT']=df_1['PERSONCOUNT'].astype('int')
df_1['PEDCOUNT']=df_1['PEDCOUNT'].astype('int')
df_1['PEDCYLCOUNT']=df_1['PEDCYLCOUNT'].astype('int')
df_1['VEHCOUNT']=df_1['VEHCOUNT'].astype('int')
df_1['INCDATE']=df_1['INCDATE'].astype('string')
df_1['INCDTTM']=df_1['INCDTTM'].astype('string')
df_1['JUNCTIONTYPE']=df_1['JUNCTIONTYPE'].astype('string')
df_1['SDOT_COLCODE']=df_1['SDOT_COLCODE'].astype('string')
df_1['SDOT_COLDESC']=df_1['SDOT_COLDESC'].astype('string')
df_1['INATTENTIONIND']=df_1['INATTENTIONIND'].astype('string')
df_1['UNDERINFL']=df_1['UNDERINFL'].astype('string')
df_1['WEATHER']=df_1['WEATHER'].astype('string')
df_1['ROADCOND']=df_1['ROADCOND'].astype('string')
df_1['LIGHTCOND']=df_1['LIGHTCOND'].astype('string')
df_1['PEDROWNOTGRNT']=df_1['PEDROWNOTGRNT'].astype('string')
df_1['SPEEDING']=df_1['SPEEDING'].astype('string')
df_1['ST_COLCODE']=df_1['ST_COLCODE'].astype('string')
df_1['ST_COLDESC']=df_1['ST_COLDESC'].astype('string')

df_1.dtypes

In [ ]:
## 1. Check about the location of heavy incidents(PERSONCOUNT over 10)
location_statistic=df_1[['Longitude','Latitude','OBJECTID','PERSONCOUNT',
                       'PEDCOUNT','PEDCYLCOUNT','VEHCOUNT']]
location_statistic
serious_incidents=location_statistic.loc[location_statistic.PERSONCOUNT>10]
serious_incidents=serious_incidents.dropna()
serious_incidents

latitude=47.64
longitude=-122.30
seattle_map=folium.Map(location=[latitude, longitude],zoom_start=11)
seattle_map


# instantiate a feature group for the incidents in the dataframe
incidents = folium.map.FeatureGroup()

# loop through the serious incidents and add each to the incidents feature group

for lat, lng, in zip(serious_incidents.Latitude, serious_incidents.Longitude):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='red',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

seattle_map.add_child(incidents)


In [ ]:
## The Exploratory data analysis
# count the incidents in each year
df_2=df_1[['INCDATE','INCDTTM','OBJECTID','PERSONCOUNT','PEDCOUNT','PEDCYLCOUNT','VEHCOUNT']]
df_2

#summarize the annual incidents
df_2['INCDATE']=pd.to_datetime(df_2['INCDATE'], format='%Y/%m/%d',errors='coerce')
annual_incidents=df_2['OBJECTID'].groupby([df_2['INCDATE'].dt.year.rename('Year')]).agg({'count'})
annual_incidents.rename(columns={'count':'Incidents'},inplace=True)
annual_incidents

# Create the bar Chart for the annual incidents
annual_incidents.plot(kind='bar',figsize=(14,8))
plt.title('The Numbers of Incidents in Seattle in last 16 years')
plt.ylabel('Numbers of the Incidents')
plt.xlabel('Year')
plt.show()

In [ ]:
# summarize the scale of incidents every year
annual_statistic_2=df_2['PEDCOUNT'].groupby([df_2['INCDATE'].dt.year.rename('Year')]).sum().to_frame(name='Pedestrians').reset_index()
annual_statistic_3=df_2['PEDCYLCOUNT'].groupby([df_2['INCDATE'].dt.year.rename('Year')]).sum().to_frame(name='Bicycles').reset_index()
annual_statistic_a=pd.concat([annual_statistic_2,annual_statistic_3],axis=1)
annual_statistic_a= annual_statistic_a.loc[:,~annual_statistic_a.columns.duplicated()]
annual_statistic_a

#create the bar chart for pedestrians and bicycles
labels=annual_statistic_a['Year']
pedestrians=annual_statistic_a['Pedestrians']
bicycles=annual_statistic_a['Bicycles']

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize=(14,8))
rects1 = ax.bar(x - width/2, pedestrians, width, label='Pedestrians')
rects2 = ax.bar(x + width/2, bicycles, width, label='Bicycles')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Numbers')
ax.set_title('Numbers of Involved Pedestrians and Bicycles')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()

In [ ]:
# The annual involved vehicles bar chart
annual_vehicle=df_2['VEHCOUNT'].groupby([df_2['INCDATE'].dt.year.rename('Year')]).sum().to_frame(name='Vehicles')
annual_vehicle
annual_vehicle.plot(kind='bar',figsize=(14,8),color='lightblue')
plt.title('The Involved Vehicles in Seattle in last 16 years')
plt.ylabel('Numbers of the Involved Vehicles')
plt.xlabel('Year')
plt.show()

In [ ]:
#create statistics for day_sessions (daily)
df_2['Hours']= pd.to_datetime(df_2['INCDTTM']).dt.hour
df_2

def f(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'

df_2['session'] = df_2['Hours'].apply(f)
df_2

day_statistics=df_2['OBJECTID'].groupby([df_2['session']]).agg({'count'})
day_statistics.rename(columns={'count':'Incidents'},inplace=True)
day_statistics

#create statistics for day_sessions
df_2['Hours']= pd.to_datetime(df_2['INCDTTM']).dt.hour
df_2

def f(x):
    if (x > 4) and (x <= 8):
        return 'Early Morning'
    elif (x > 8) and (x <= 12 ):
        return 'Morning'
    elif (x > 12) and (x <= 16):
        return'Noon'
    elif (x > 16) and (x <= 20) :
        return 'Eve'
    elif (x > 20) and (x <= 24):
        return'Night'
    elif (x <= 4):
        return'Late Night'

df_2['session'] = df_2['Hours'].apply(f)
df_2

day_statistics=df_2['OBJECTID'].groupby([df_2['session']]).agg({'count'})
day_statistics.rename(columns={'count':'Incidents'},inplace=True)
day_statistics

Day_pedestrian=df_2['PERSONCOUNT'].groupby([df_2['session']]).sum().to_frame(name='Pedestrians').reset_index()
Day_bicycle=df_2['PEDCYLCOUNT'].groupby([df_2['session']]).sum().to_frame(name='Bicycles').reset_index()
Day_vehicle=df_2['VEHCOUNT'].groupby([df_2['session']]).sum().to_frame(name='Vehicles').reset_index()
Day_statistics=pd.concat([Day_pedestrian,Day_bicycle,Day_vehicle],axis=1)
Day_statistics= Day_statistics.loc[:,~Day_statistics.columns.duplicated()]
Day_statistics

#create the bar chart for pedestrians and bicycles
label1=Day_statistics['session']
pedestrians=Day_statistics['Pedestrians']
bicycles=Day_statistics['Bicycles']
vehicles=Day_statistics['Vehicles']

x = np.arange(len(label1))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(14,8))
rects1 = ax.bar(x - 0.2, pedestrians, width, label='Pedestrians')
rects2 = ax.bar(x , bicycles, width, label='Bicycles')
rects3 = ax.bar(x + 0.2, vehicles, width, label='Vehicles')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Numbers')
ax.set_title('Numbers of Involved Pedestrians and Bicycles in different time sessions in one day')
ax.set_xticks(x)
ax.set_xticklabels(label1)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()

In [ ]:
day_statistics.plot(kind='line',figsize=(14,8),color='orange')
plt.title('The Incident Sessions in Seattle')
plt.ylabel('The Numbers of Incidents')
plt.xlabel('Time Session')

In [ ]:
#### Seasonal statistics
df_2['Month']= pd.to_datetime(df_2['INCDATE']).dt.month
df_2

def g(x):
    if (x >=3) and (x < 6):
        return 'Spring'
    elif (x >=6) and (x < 9 ):
        return 'Summer'
    elif (x >=9) and (x < 12):
        return'Autumn'
    else:
        return'Winter'

df_2['Season'] = df_2['Month'].apply(g)
df_2

season_statistics=df_2['OBJECTID'].groupby([df_2['Season']]).agg({'count'})
season_statistics.rename(columns={'count':'Incidents'},inplace=True)
season_statistics_c=season_statistics.reset_index()
season_statistics_c

#horizontal bar chart for each season

season = season_statistics_c['Season']
y_pos = np.arange(len(season))
incidents = season_statistics_c['Incidents']

plt.barh(y_pos, incidents, align='center', alpha=0.5)
plt.yticks(y_pos, season)
plt.xlabel('The numbers of incidents')
plt.title('The numbers of incidents in each season')

plt.show()

season_pedestrian=df_2['PERSONCOUNT'].groupby([df_2['Season']]).sum().to_frame(name='Pedestrians').reset_index()
season_bicycle=df_2['PEDCYLCOUNT'].groupby([df_2['Season']]).sum().to_frame(name='Bicycles').reset_index()
season_vehicle=df_2['VEHCOUNT'].groupby([df_2['Season']]).sum().to_frame(name='Vehicles').reset_index()
season_statistics_2=pd.concat([season_pedestrian,season_bicycle,season_vehicle],axis=1)
season_statistics_2= season_statistics_2.loc[:,~season_statistics_2.columns.duplicated()]
season_statistics_2

#create the bar chart for pedestrians and bicycles
label2=season_statistics_2['Season']
pedestrians=season_statistics_2['Pedestrians']
bicycles=season_statistics_2['Bicycles']
vehicles=season_statistics_2['Vehicles']

x = np.arange(len(label2))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(14,8))
rects1 = ax.bar(x - 0.2, pedestrians, width, label='Pedestrians')
rects2 = ax.bar(x , bicycles, width, label='Bicycles')
rects3 = ax.bar(x + 0.2, vehicles, width, label='Vehicles')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Numbers')
ax.set_title('Numbers of Involved Pedestrians and Bicycles in different seasons')
ax.set_xticks(x)
ax.set_xticklabels(label2)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()


In [ ]:
# Create analysis for the other attributes
df_3=df_1[['OBJECTID','ADDRTYPE','SEVERITYCODE','SEVERITYDESC','COLLISIONTYPE','JUNCTIONTYPE','SDOT_COLCODE','SDOT_COLDESC','PERSONCOUNT',
           'PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','INATTENTIONIND','UNDERINFL','WEATHER','ROADCOND','LIGHTCOND',
          'PEDROWNOTGRNT','SPEEDING']]
df_3

In [ ]:
#1. Add_type analysis
addtype=df_3['OBJECTID'].groupby([df_3['ADDRTYPE']]).agg({'count'})
addtype.rename(columns={'count':'Incidents'},inplace=True)
addtype=addtype.reset_index()
addtype

# pie chart for add_type analysis

label4=addtype['ADDRTYPE']
sizes=addtype['Incidents']
explode=(0.2,0.2,0.2) #only "explode" the 1st slice "block"


fig1,ax1=plt.subplots(figsize=(14,10))
ax1.pie(sizes,explode=explode,labels=label4,shadow=True,autopct='%1.1f%%',startangle=90,textprops={'fontsize': 16})
ax1.axis('equal')

plt.show()


In [ ]:
#2. add analysis for incident scale
addtype_pedestrians=df_3['PERSONCOUNT'].groupby([df_3['ADDRTYPE']]).sum().to_frame(name='Pedestrians').reset_index()
addtype_bicycle=df_3['PEDCYLCOUNT'].groupby([df_3['ADDRTYPE']]).sum().to_frame(name='Bicycles').reset_index()
addtype_vehicle=df_3['VEHCOUNT'].groupby([df_3['ADDRTYPE']]).sum().to_frame(name='Vehicles').reset_index()
addtype_statistics=pd.concat([addtype_pedestrians,addtype_bicycle,addtype_vehicle],axis=1)
addtype_statistics= addtype_statistics.loc[:,~addtype_statistics.columns.duplicated()]
addtype_statistics


#create the bar chart for add analysis
#create the bar chart for pedestrians and bicycles
label_add=addtype_statistics['ADDRTYPE']
pedestrians_add=addtype_statistics['Pedestrians']
bicycles_add=addtype_statistics['Bicycles']
vehicles_add=addtype_statistics['Vehicles']

x = np.arange(len(label_add))  # the label locations
width = 0.2  # the width of the bars

fig, ax = plt.subplots(figsize=(14,8))
rects1 = ax.bar(x - 0.2, pedestrians_add, width, label='Pedestrians')
rects2 = ax.bar(x , bicycles_add, width, label='Bicycles')
rects3 = ax.bar(x + 0.2, vehicles_add, width, label='Vehicles')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Numbers',size=14)
ax.set_title('Numbers of Involved Pedestrians, Bicycles and Vehcile in different location')
ax.set_xticks(x)
ax.set_xticklabels(label_add,size=14)
ax.legend()

def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

fig.tight_layout()

plt.show()


In [ ]:
#2. SEVERITYDESC analysis
SEVERITYDESC_1=df_3['OBJECTID'].groupby([df_3['SEVERITYDESC']]).agg({'count'})
SEVERITYDESC_1.rename(columns={'count':'Incidents'},inplace=True)
SEVERITYDESC_1=SEVERITYDESC_1.reset_index()
SEVERITYDESC_1

# pie chart for add_type analysis

label_seve=SEVERITYDESC_1['SEVERITYDESC']
sizes_seve=SEVERITYDESC_1['Incidents']
explode=(0,0.2) #only "explode" the 1st slice "block"


fig2,ax2=plt.subplots(figsize=(14,10))
ax2.pie(sizes_seve,explode=explode,labels=label_seve,shadow=True,autopct='%1.1f%%',startangle=90,textprops={'fontsize': 16})
ax2.axis('equal')

plt.show()

In [ ]:
combine_1=df_3['OBJECTID'].groupby([df_3['SEVERITYDESC'],df_3['ADDRTYPE']]).agg({'count'})
combine_1=combine_1.reset_index()
combine_1

In [ ]:
combine_pedestrians=df_3['PERSONCOUNT'].groupby([df_3['SEVERITYDESC'],df_3['ADDRTYPE']]).sum().to_frame(name='Pedestrians').reset_index()

combine_bicycle=df_3['PEDCYLCOUNT'].groupby([df_3['SEVERITYDESC'],df_3['ADDRTYPE']]).sum().to_frame(name='Bicycles').reset_index()

combine_vehicle=df_3['VEHCOUNT'].groupby([df_3['SEVERITYDESC'],df_3['ADDRTYPE']]).sum().to_frame(name='Vehicles').reset_index()

combine_statistics=pd.concat([combine_pedestrians,combine_bicycle,combine_vehicle],axis=1)

combine_statistics= combine_statistics.loc[:,~combine_statistics.columns.duplicated()]
combine_statistics

In [ ]:
combine=pd.concat([combine_1,combine_statistics],axis=1)
combine= combine.loc[:,~combine.columns.duplicated()]
combine

In [ ]:
combine.to_excel(r'/Users/Lionpf/Desktop/Coursera_Capstone/Coursera_Capstone/Report\combine.xlsx', index = False)

In [ ]:
weather_1=df_3['OBJECTID'].groupby([df_3['WEATHER']]).agg({'count'})
weather_1

weather_1.plot(kind='bar',figsize=(14,8),color='slateblue',fontsize=14)
plt.title('The numbers of incidents in different weather conditions',size=14)
plt.ylabel('Numbers of incidents',fontsize=14)
plt.xlabel('Weather condition',fontsize=14)
plt.show()

In [ ]:
road_1=df_3['OBJECTID'].groupby([df_3['ROADCOND']]).agg({'count'})
road_1

road_1.plot(kind='bar',figsize=(14,8),color='lightgreen',fontsize=14)
plt.title('The numbers of incidents in different road conditions',size=14)
plt.ylabel('Numbers of incidents',fontsize=14)
plt.xlabel('Road condition',fontsize=14)
plt.show()

In [ ]:
light_1=df_3['OBJECTID'].groupby([df_3['LIGHTCOND']]).agg({'count'})
light_1

light_1.plot(kind='bar',figsize=(14,8),color='lightgrey',fontsize=14)
plt.title('The numbers of incidents in different light conditions',size=14)
plt.ylabel('Numbers of incidents',fontsize=14)
plt.xlabel('Light condition',fontsize=14)
plt.show()

In [ ]:
UNDERINFL_1=df_3['OBJECTID'].groupby([df_3['UNDERINFL']]).agg({'count'})
UNDERINFL_1

In [ ]:
COLLISIONTYPE_1=df_3['OBJECTID'].groupby([df_3['SEVERITYDESC'],df_3['COLLISIONTYPE']]).agg({'count'})
COLLISIONTYPE_1

In [ ]:
df_4=df_1[['INCDATE','INCDTTM','OBJECTID','ADDRTYPE','SEVERITYCODE','SEVERITYDESC','COLLISIONTYPE','JUNCTIONTYPE','SDOT_COLCODE','SDOT_COLDESC','PERSONCOUNT',
           'PEDCOUNT','PEDCYLCOUNT','VEHCOUNT','WEATHER','ROADCOND','LIGHTCOND']]
df_4

In [ ]:
df_4['INCDATE']=pd.to_datetime(df_4['INCDATE'], format='%Y/%m/%d',errors='coerce')

In [ ]:
incident_1=df_4['OBJECTID'].groupby([df_4['INCDATE'].dt.year.rename('Year')]).agg({'count'})
incident_1=incident_1.reset_index()
incident_1

In [ ]:
statistic_1=df_4['PEDCOUNT'].groupby([df_4['INCDATE'].dt.year.rename('Year')]).sum().to_frame(name='Pedestrians').reset_index()
statistic_2=df_4['PEDCYLCOUNT'].groupby([df_4['INCDATE'].dt.year.rename('Year')]).sum().to_frame(name='Bicycles').reset_index()
statistic_3=df_4['VEHCOUNT'].groupby([df_4['INCDATE'].dt.year.rename('Year')]).sum().to_frame(name='Vehicles').reset_index()
statistics=pd.concat([statistic_1,statistic_2,statistic_3],axis=1)
statistics= statistics.loc[:,~statistics.columns.duplicated()]
statistics

In [ ]:
incident_2=df_4['OBJECTID'].groupby([df_4['INCDATE'].dt.year.rename('Year'),df_4['ADDRTYPE']]).agg({'count'})
incident_2=incident_2.reset_index()
incident_2

In [ ]:
severity=df_4['OBJECTID'].groupby([df_4['SEVERITYCODE']]).agg({'count'})
severity